## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Pochutla,MX,15.7432,-96.4661,82.22,53,13,8.05,few clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,36.07,92,86,9.64,overcast clouds
2,2,Hun,LY,29.1268,15.9477,79.63,16,84,21.54,broken clouds
3,3,Lima,PE,-12.0432,-77.0282,65.39,61,0,17.27,clear sky
4,4,Punta Arenas,CL,-53.1500,-70.9167,42.91,87,40,8.05,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for you trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))


What is the minimum temperature you would like for you trip?70
What is the maximum temperature you would like for your trip?85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp)&\
                                      (city_data_df['Max Temp']>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Pochutla,MX,15.7432,-96.4661,82.22,53,13,8.05,few clouds
2,2,Hun,LY,29.1268,15.9477,79.63,16,84,21.54,broken clouds
7,7,Micheweni,TZ,-4.9667,39.8333,79.25,82,100,14.27,light rain
9,9,Bantayan,PH,11.2000,123.7333,81.27,79,100,11.36,overcast clouds
11,11,Navirai,BR,-23.0650,-54.1906,72.05,74,49,3.98,scattered clouds
13,13,Vaini,TO,-21.2000,-175.2000,82.56,89,75,9.22,broken clouds
14,14,Saint-Philippe,RE,-21.3585,55.7679,74.79,78,15,8.14,few clouds
20,20,Rikitea,PF,-23.1203,-134.9692,76.57,73,73,10.49,moderate rain
29,29,Avarua,CK,-21.2078,-159.7750,84.25,74,20,10.36,few clouds
44,44,Atuona,PF,-9.8000,-139.0333,79.05,76,8,11.43,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Pochutla,MX,82.22,few clouds,15.7432,-96.4661,
2,Hun,LY,79.63,broken clouds,29.1268,15.9477,
7,Micheweni,TZ,79.25,light rain,-4.9667,39.8333,
9,Bantayan,PH,81.27,overcast clouds,11.2000,123.7333,
11,Navirai,BR,72.05,scattered clouds,-23.0650,-54.1906,
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
14,Saint-Philippe,RE,74.79,few clouds,-21.3585,55.7679,
20,Rikitea,PF,76.57,moderate rain,-23.1203,-134.9692,
29,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,
44,Atuona,PF,79.05,clear sky,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location']=f'{lat},{lng}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping.')
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Pochutla,MX,82.22,few clouds,15.7432,-96.4661,Hotel Posada San Jose
2,Hun,LY,79.63,broken clouds,29.1268,15.9477,Alrwasi Hotel
9,Bantayan,PH,81.27,overcast clouds,11.2000,123.7333,Bantayan Island Nature Park and Resort
11,Navirai,BR,72.05,scattered clouds,-23.0650,-54.1906,Dubai Hotel - Também sua casa!
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
652,Pokhara,NP,70.75,overcast clouds,28.2333,83.9833,OYO 170 Hotel View Point
653,Niquero,CU,77.76,clear sky,20.0431,-77.5828,Islazul Niquero
661,Chimore,BO,70.07,clear sky,-16.9833,-65.1333,Bichito
662,Auki,SB,84.60,overcast clouds,-8.7676,160.7034,Auki Motel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}<dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))